In [6]:
import requests
import pandas as pd
import openmeteo_requests
import requests_cache
import pandas as pd
import httpx
from retry_requests import retry
from bs4 import BeautifulSoup

In [7]:
infos_dict = {
'scheduled' : {
'url': "https://www.portodesantos.com.br/informacoes-operacionais/operacoes-portuarias/navegacao-e-movimento-de-navios/atracacoes-programadas/",
'columns':['Date','Hour','ETA','Place','Ship','IMO','Cargo','Event','Voyage','DUV'],
'df' : None
},

'foreseen_cargo' : {
'url':"https://www.portodesantos.com.br/informacoes-operacionais/operacoes-portuarias/navegacao-e-movimento-de-navios/navios-esperados-carga/",
'columns':['Ship','Flag','Draft','Nav','Arrival','Notice','Office','Operation','Goods','Weight','Voyage','DUV','P','Terminal','IMO'],
'df' : None
},

'foreseen_cruise' : {
'url':"https://www.portodesantos.com.br/informacoes-operacionais/operacoes-portuarias/navegacao-e-movimento-de-navios/navios-esperados-passageiros/",
'columns':['Ship','Flag','Draft','Nav','Arrival','Notice','Office','Voyage','DUV','P','Terminal'],
'df' : None
},

'berthage_ships' : {
'url':"https://www.portodesantos.com.br/informacoes-operacionais/operacoes-portuarias/navegacao-e-movimento-de-navios/navios-fundeados/",
'columns':['Ship','Flag','Draft','Nav','Arrival','Notice','Office','Operat','Type','Weight','Voyage','P','Terminal'],
'df' : None
},

'berthed_ships' : {
'url':"https://www.portodesantos.com.br/informacoes-operacionais/operacoes-portuarias/navegacao-e-movimento-de-navios/atracados-porto-terminais/",
'columns':['Place','Ship','Cargo','Unload','Load'],
'df' : None
},
}

In [8]:
for key, value in infos_dict.items():

    table = None
    op = None

    url = value['url']
    columns_df = value['columns']
    
    data = requests.get(url, verify=False).text
    soup = BeautifulSoup(data, 'html.parser')

    if key != 'berthed_ships':
        table = soup.find_all('tbody')

    else:
        table = soup.find('table', {'id':'atracados'})

    df = pd.DataFrame(columns = columns_df)

    for i in range(len(table)):

        if key != 'berthed_ships':
            op = table[i].find_all('tr')

        else:
            op = table.tbody.find_all('tr')

        for row in op:

            columns = row.find_all('td')

            if(columns != []):

                dict_merge = {}

                for j in range(len(columns_df)):

                    if key != 'berthed_ships':

                        row = {
                            columns_df[j]: columns[j].text.strip()
                        }
                    
                    else:
                                                
                        columns_index = (0,1,6,7,8)

                        index_column = columns_index[j]

                        row = {
                            columns_df[j]: columns[index_column].text.strip()
                        }

                    dict_merge = {**dict_merge, **row}

                df = pd.concat([df, pd.DataFrame([dict_merge])], axis = 0, ignore_index=True) 
    
    value['df'] = df 


c:\Users\João Vitor\Desktop\Projetos\Santos_Port_n_Vessels\venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.portodesantos.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\João Vitor\Desktop\Projetos\Santos_Port_n_Vessels\venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.portodesantos.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\João Vitor\Desktop\Projetos\Santos_Port_n_Vessels\venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.portodesantos.com.br'. Adding certificate verification is strongly advised. See: https

In [9]:
dim_ships =  pd.DataFrame(columns = ['Ship','Flag'])

for key, value in infos_dict.items():

    if 'Flag' not in infos_dict[key]['df'].columns:

        infos_dict[key]['df']['Flag'] = None
    
    if 'IMO' not in infos_dict[key]['df'].columns:

        infos_dict[key]['df']['IMO'] = None

    dim_ships = pd.concat([dim_ships,pd.DataFrame(infos_dict[key]['df'].loc[:,['Ship','Flag','IMO']])], axis = 0, ignore_index=True) 
    dim_ships.drop_duplicates(subset='Ship',inplace=True)
    

In [10]:
dim_ships

,Ship,Flag,IMO
0,COSCO SHIPPING ARGENTINA,None,9945849
1,INDUSTRIAL MOMENTUM,None,9534432
2,ALKYONI,None,9624055
3,ANDROMEDA,None,9705110
4,AS SABINE,None,9813802
...,...,...,...
336,PERSISTENCE DIVA,None,None
337,TANABATA,None,None
338,PHOENIX OCEAN,None,None
342,DESERT PIONEER,None,None


In [11]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": -23.9608,
	"longitude": -46.3336,
	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation_probability", "precipitation", "rain", "pressure_msl", "surface_pressure", "visibility", "evapotranspiration", "wind_speed_80m", "wind_speed_120m", "wind_direction_80m", "wind_direction_120m"],
	"daily": ["sunrise", "sunset", "uv_index_max"],
	"timezone": "America/Sao_Paulo",
	"past_days": 2
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation_probability = hourly.Variables(3).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
hourly_rain = hourly.Variables(5).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(6).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(7).ValuesAsNumpy()
hourly_visibility = hourly.Variables(8).ValuesAsNumpy()
hourly_evapotranspiration = hourly.Variables(9).ValuesAsNumpy()
hourly_wind_speed_80m = hourly.Variables(10).ValuesAsNumpy()
hourly_wind_speed_120m = hourly.Variables(11).ValuesAsNumpy()
hourly_wind_direction_80m = hourly.Variables(12).ValuesAsNumpy()
hourly_wind_direction_120m = hourly.Variables(13).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["precipitation_probability"] = hourly_precipitation_probability
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["visibility"] = hourly_visibility
hourly_data["evapotranspiration"] = hourly_evapotranspiration
hourly_data["wind_speed_80m"] = hourly_wind_speed_80m
hourly_data["wind_speed_120m"] = hourly_wind_speed_120m
hourly_data["wind_direction_80m"] = hourly_wind_direction_80m
hourly_data["wind_direction_120m"] = hourly_wind_direction_120m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_sunrise = daily.Variables(0).ValuesAsNumpy()
daily_sunset = daily.Variables(1).ValuesAsNumpy()
daily_uv_index_max = daily.Variables(2).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["uv_index_max"] = daily_uv_index_max

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

Coordinates -24.0°N -46.375°E
Elevation 6.0 m asl
Timezone b'America/Sao_Paulo' b'-03'
Timezone difference to GMT+0 -10800 s
                         date  temperature_2m  relative_humidity_2m  \
0   2025-01-01 03:00:00+00:00       24.767000                  86.0   
1   2025-01-01 04:00:00+00:00       24.167000                  89.0   
2   2025-01-01 05:00:00+00:00       23.767000                  91.0   
3   2025-01-01 06:00:00+00:00       23.717001                  91.0   
4   2025-01-01 07:00:00+00:00       23.417000                  91.0   
..                        ...             ...                   ...   
211 2025-01-09 22:00:00+00:00       24.067001                  74.0   
212 2025-01-09 23:00:00+00:00       23.817001                  77.0   
213 2025-01-10 00:00:00+00:00       23.567001                  79.0   
214 2025-01-10 01:00:00+00:00       23.217001                  81.0   
215 2025-01-10 02:00:00+00:00       22.817001                  83.0   

     dew_point_2m  pre